# Import

In [ ]:
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
from mesa.visualization.modules import CanvasGrid, ChartModule
from mesa.visualization.ModularVisualization import ModularServer

import warnings
import random

# Init model

In [ ]:
GLOBAL_METABOLISM = 1
GLOBAL_VISION = 3
GLOBAL_MAX_SUGAR = 4
GLOBAL_INIT_AGENT_SUGAR = 20

# 定义SugarAgent类
class SugarAgent(Agent):
    def __init__(self, unique_id, model, pos, sugar):
        super().__init__(unique_id, model)  # 显式调用父类的初始化方法
        self.pos = pos
        self.sugar = sugar
        self.metabolism = GLOBAL_METABOLISM
        self.vision = GLOBAL_VISION
    
    def step(self):
        self.move()
        self.eat()
        self.metabolize()
        
    def move(self):
        neighborhood = self.model.grid.get_neighborhood(self.pos, moore=True, radius=self.vision)
        # 只考虑有糖的位置
        valid_cells = [cell for cell in neighborhood if any(isinstance(obj, SugarPatch) for obj in self.model.grid.get_cell_list_contents(cell))]

        if not valid_cells:
            return  # No available moves, stay in place

        # 找到最大糖量
        max_sugar = max(max(obj.sugar for obj in self.model.grid.get_cell_list_contents(pos) if isinstance(obj, SugarPatch)) for pos in valid_cells)

        # 找出所有糖量等于最大糖量的单元格
        max_sugar_cells = [pos for pos in valid_cells if any(obj.sugar == max_sugar for obj in self.model.grid.get_cell_list_contents(pos) if isinstance(obj, SugarPatch))]

        # 随机选择一个最大糖量的单元格
        new_position = self.random.choice(max_sugar_cells)

        self.model.grid.move_agent(self, new_position)



    def eat(self):
        patch = self.model.grid.get_cell_list_contents([self.pos])[0]
        if isinstance(patch, SugarPatch):
            self.sugar += patch.sugar
            patch.sugar = 0
    
    def metabolize(self):
        self.sugar -= self.metabolism
        if self.sugar <= 0:
            self.model.grid.remove_agent(self)
            self.model.schedule.remove(self)

# 定义SugarPatch类
class SugarPatch(Agent):
    def __init__(self, unique_id, model, pos, max_sugar):
        super().__init__(unique_id, model)  # 显式调用父类的初始化方法
        self.pos = pos
        self.max_sugar = max_sugar
        self.sugar = random.randint(0,max_sugar)
    
    def step(self):
        self.regrow()
        
    def regrow(self):
        if self.sugar < self.max_sugar:
            self.sugar += 1

# 定义Sugarscape模型
class Sugarscape(Model):
    def __init__(self, width, height, initial_agents):
        super().__init__()
        self.grid = MultiGrid(width, height, torus=False) # torus means circle
        # Initialize the current_id property to ensure that the next_id method can work properly
        self.current_id = 0

        # Randomly shuffle the order of agents and activate each agent once.
        self.schedule = RandomActivation(self)
        """def step(self):
        self.model.random.shuffle(self.agents)
        for agent in self.agents:
            agent.step()
        """

        # Collect data at model level and agent level, for now it just use RandomActivation to collect the 
        # number of agents
        self.datacollector = DataCollector(
        {
            "AgentCount": lambda m: sum(1 for a in m.schedule.agents if isinstance(a, SugarAgent))
        }
        )

        # 初始化糖资源
        # self.grid.coord_iter() is a method provided by MultiGrid that is used to iterate over each 
        # cell in the grid.
        # contents: represents the contents of the cell
        for (contents, pos) in self.grid.coord_iter():
            max_sugar = GLOBAL_MAX_SUGAR
            patch = SugarPatch(self.next_id(), self, pos, max_sugar)
            # Suppress specific warnings
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", UserWarning)
                self.grid.place_agent(patch, pos)
            self.schedule.add(patch)
            

        # 初始化代理
        available_positions = [(x, y) for x in range(self.grid.width) for y in range(self.grid.height)]
        self.random.shuffle(available_positions)

        for i in range(initial_agents):
            if not available_positions:
                raise ValueError("No available positions to place new agents.")

            pos = available_positions.pop()
            sugar = GLOBAL_INIT_AGENT_SUGAR
            agent = SugarAgent(self.next_id(), self, pos, sugar)
            # Suppress specific warnings
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", UserWarning)
                self.grid.place_agent(agent, pos)
            self.schedule.add(agent)
        
    def step(self):
        self.schedule.step()
        self.datacollector.collect(self)
"""
# local运行模型
model = Sugarscape(50, 50, 100)
for i in range(15):
    model.step()

# 收集数据
data = model.datacollector.get_model_vars_dataframe()
print(data)
"""
# Browser-based visual interface
def agent_portrayal(agent):
    if isinstance(agent, SugarAgent):
        portrayal = {
            "Shape": "circle",
            "Filled": "true",
            "r": 0.5,
            "Color": "blue",
            "Layer": 1,
            "text": agent.sugar,
            "text_color": "white",
        }
    elif isinstance(agent, SugarPatch):
        portrayal = {
            "Shape": "rect",
            "w": 1,
            "h": 1,
            "Filled": "true",
            "Color": "green",
            "Layer": 0,
            "text": agent.sugar,
            "text_color": "black",
        }
    return portrayal

grid = CanvasGrid(agent_portrayal, 50, 50, 500, 500)
chart = ChartModule([{"Label": "AgentCount", "Color": "Black"}])



# Run

In [ ]:
server = ModularServer(Sugarscape,
                       [grid, chart],
                       "Sugarscape Model",
                       {"width": 50, "height": 50, "initial_agents": 100})

server.port = 8532
server.launch()